In [1]:
# This automates removal of cosmic rays from SDSS red and blue single-epoch spectra,
# as required by Young Sun

# Created 2022 Dec. 3 by E.S.

In [2]:
'''
Order of operations:

1.) Based on number of single-epoch spectra
    - If 1 spectrum only, ignore for now
    - If 2 spectra only, do a sigma-clipping and identify anomalies *upward* (with a window) 
    - If >=3 spectra, find median spectrum and identify outliers (with a window) 
2.) TBD
'''

'\nOrder of operations:\n\n1.) Based on number of single-epoch spectra\n    - If 1 spectrum only, ignore for now\n    - If 2 spectra only, do a sigma-clipping and identify anomalies *upward* (with a window) \n    - If >=3 spectra, find median spectrum and identify outliers (with a window) \n2.) TBD\n'

In [11]:
import pandas as pd
import numpy as np
import glob
import sys
import os
import matplotlib.pyplot as plt
from astropy.stats import sigma_clip

%matplotlib qt

In [4]:
# top-level directory for SDSS spectra cosmic ray removal
stem_raw_single_epoch = "/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/"+\
                        "01_separated_and_interpolated/"

# find individual file names
file_list = glob.glob(stem_raw_single_epoch + "*.csv")
# find all parent names (i.e., one name for each target, whether or not multiepoch observations were made)
parent_list = list(set([i.split("_g0")[0] for i in file_list]))

# find individual file names
file_list_red = glob.glob(stem_raw_single_epoch + "*color_red.csv")
# find all parent names (i.e., one name for each target, whether or not multiepoch observations were made)
parent_list_red = list(set([i.split("_g0")[0] for i in file_list_red]))

In [5]:
def flag_regions(df_empir_pass,df_avg_pass,df_median_pass,sigma_choice=1):
    '''
    Flag points based on their deviation from the average spectrum
    
    INPUTS:
    df_empir_pass: dataframe of empirical spectrum
    df_avg_pass: dataframe of average spectrum
    df_median_pass: dataframe of median spectrum
    sigma_choice: threshold for clipping
    '''
    
    # initialize DataFrame to return
    masked_spec = df_empir_pass.copy(deep=True)
    #masked_spec["flux_masked_1"] = masked_spec["flux"]
    
    # take difference between empirical spectrum and the AVERAGE of the AVERAGE AND MEDIAN spectrum
    # (note this preserves sign information, and (if only 2 spectra are being compared) protects against 
    # misidentification of a cosmic ray in 1 spectrum when the ray is actually in the other)
    #initialize DataFrame for taking an average of some kind
    standin_df = df_avg_pass.copy(deep=True)
    standin_df["median_flux"] = df_median_pass["median_flux"]
    # remove column of wavelengths
    print(standin_df.keys())
    standin_df = standin_df.drop(labels=["wavel"],axis=1)
    # find the mean of a mean and a median
    standin_df["mean_of_stuff"] = standin_df.mean(axis=1) # average of the columns (superfluous?)
    
    #avg_flux = np.expand_dims(df_avg_pass["avg_flux"].values,axis=1)
    #median_flux = np.expand_dims(df_median_pass["median_flux"].values,axis=1)
    #print(np.expand_dims(avg_flux,axis=0).shape)
    #print(median_flux.shape)
    #mean_median_combo = np.mean(avg_flux,median_flux)
    
    # difference between empirical and median 
    # masked_spec["diff"] = np.subtract(df_empir_pass["flux"],standin_df["mean_of_stuff"])
    masked_spec["diff"] = np.subtract(df_empir_pass["flux"],standin_df["median_flux"])
    
    #plt.plot(masked_spec["diff"])
    #plt.show()
    
    # mask deviant points
    # logic: is positive difference between empirical and median beyond error bounds?
    error_bound = sigma_choice*np.nanstd(masked_spec["diff"])
    logic_1 = np.greater(masked_spec["diff"],error_bound)
    masked_spec["flux_flag_1"] = logic_1 # flag these points as suspect
    
    return masked_spec, error_bound

In [12]:
# loop over each parent FITS file
for t in range(0,1):#len(parent_list)):

    matching_all = list(filter(lambda x: parent_list[t] in x, file_list))
    matching_red = list(filter(lambda x: "color_red" in x, matching_all))
    matching_blue = list(filter(lambda x: "color_blue" in x, matching_all))
    
    #print(matching_all)
    #print(matching_red)
    #print(matching_blue)
    
    # keep the red and blue parts separate
    for color_num in range(0,2):
        
        if color_num==0:
            matching = matching_red
        elif color_num==1:
            matching = matching_blue
        #print(matching)

        #print("-------------------------")

        if (len(matching) == 1):

            print("Only one match found for this color:")
            print(matching)

        elif (len(matching) >= 2):


            # dictionary to hold dataframes
            d = {}

            # intialize aggregate array to contain all fluxes
            df_dummy = pd.read_csv(matching[0], names=["wavel","flux","noise"], delim_whitespace=False, skiprows=1)
            aggregate_flux_array = np.nan*np.ones((len(df_dummy),len(matching)))

            # collect spectra in single dictionary
            for p in range(0,len(matching)):

                # read in one of the matches
                df_single_p = pd.read_csv(matching[p], names=["wavel","flux","noise"], delim_whitespace=False, skiprows=1)

                #plt.plot(df_single_p["wavel"],df_single_p["flux"])

                # sanity check that wavelength abcissa are the same among all the matches
                if p==0:
                    # for checking wavel abcissa is same
                    wavel_initial = df_single_p["wavel"].values
                else:
                    #print(df_single_p["wavel"])
                    #print(wavel_initial)
                    #print(len(np.setdiff1d(df_single_p["wavel"].values,wavel_initial)))
                    if len(np.setdiff1d(df_single_p["wavel"].values,wavel_initial) >= 1):
                        print("Hey, the wavelength abcissas are not the same!")
                        sys.exit()

                # put fluxes into aggregate array
                aggregate_flux_array[:,p] = df_single_p["flux"].values


            # take mean flux of all the spectra
            mean_flux_array = np.mean(aggregate_flux_array,axis=1)

            # cast mean spectrum data to a DataFrame
            df_mean = pd.DataFrame(mean_flux_array,columns=["avg_flux"])
            df_mean["wavel"] = df_single_p["wavel"] # uses last spectrum read in

            # include median flux too (important for identifying cosmic rays when only 2 spectra are compared)
            median_flux_array = np.median(aggregate_flux_array,axis=1)
            df_median = pd.DataFrame(median_flux_array,columns=["median_flux"])
            df_median["wavel"] = df_single_p["wavel"] # uses last spectrum read in
            #mean_flux_array["median_flux"] = pd.Series(median_flux_array.tolist())

            for p in range(0,len(matching)):
                
                #print("len(matching)", len(matching))
                #print(matching)

                # test each empirical spectrum against the mean, and flag points
                df_single_p = pd.read_csv(matching[p], names=["wavel","flux","noise"], delim_whitespace=False, skiprows=1)
                flagged_empirical, limit = flag_regions(
                                                        df_empir_pass = df_single_p,
                                                        df_avg_pass = df_mean,
                                                        df_median_pass = df_median,
                                                        sigma_choice=3
                                                        )


                # if cosmic ray appears to be in an absorption line, discard the spectrum
                ## ## TBD


                #plt.plot(wavel_initial,mean_flux_array,linestyle="--",color="k")
                #plt.show()
                #plt.clf()
            
                ## ## CONTINUE HERE; MAKE SURE FLAGS ARE GOOD FOR EACH CHILD SPECTRUM
                #plt.clf()
                #fig = plt.figure(figsize=(24,10))
                plt.plot(flagged_empirical["wavel"],
                         np.multiply(100,flagged_empirical["flux_flag_1"]),color="gray",alpha=0.5,label="flag")
                #.axvline(x=0, ymin=0, ymax=1

                # plot mean flux
                plt.plot(df_mean["wavel"],np.add(df_mean["avg_flux"],0.2),label="mean")
                # plot median flux
                plt.plot(df_mean["wavel"],np.add(df_median["median_flux"],0.2),label="median")
                # plot empirical flux
                plt.plot(flagged_empirical["wavel"],flagged_empirical["flux"],label="empirical")
                # plot flux minus mean flux
                plt.plot(flagged_empirical["wavel"],flagged_empirical["diff"],label="diff")
                #plt.plot(df_single_p["wavel"].where(test["flux_flag_1"] == True),
                #             df_single_p["flux"].where(test["flux_flag_1"] == True),
                #         label="flagged",color="k",linewidth=4)
                plt.plot([3900,5000],[limit,limit],linestyle="--")
                plt.title(str(os.path.basename(matching[p]))+ " match number" + str(p))
                plt.legend(loc="lower right")
                file_name_write = "plot_" + str(os.path.basename(matching[p])) + ".png"
                #plt.savefig(file_name_write, facecolor="white", edgecolor='white')
                print("Wrote", file_name_write)
                plt.show()

Index(['avg_flux', 'wavel', 'median_flux'], dtype='object')
Wrote plot_spec-0312-51689-0391_g000_color_red.csv.png
Index(['avg_flux', 'wavel', 'median_flux'], dtype='object')
Wrote plot_spec-0312-51689-0391_g004_color_red.csv.png
Index(['avg_flux', 'wavel', 'median_flux'], dtype='object')
Wrote plot_spec-0312-51689-0391_g003_color_red.csv.png
Index(['avg_flux', 'wavel', 'median_flux'], dtype='object')
Wrote plot_spec-0312-51689-0391_g002_color_red.csv.png
Index(['avg_flux', 'wavel', 'median_flux'], dtype='object')
Wrote plot_spec-0312-51689-0391_g001_color_red.csv.png
Index(['avg_flux', 'wavel', 'median_flux'], dtype='object')
Wrote plot_spec-0312-51689-0391_g000_color_blue.csv.png
Index(['avg_flux', 'wavel', 'median_flux'], dtype='object')
Wrote plot_spec-0312-51689-0391_g001_color_blue.csv.png
Index(['avg_flux', 'wavel', 'median_flux'], dtype='object')
Wrote plot_spec-0312-51689-0391_g004_color_blue.csv.png
Index(['avg_flux', 'wavel', 'median_flux'], dtype='object')
Wrote plot_spec-0